In [5]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from __future__ import unicode_literals  

import os
import math
import numpy as np
import pandas as pd
import sklearn.linear_model as linear_model

import scipy
import sklearn

import influence.experiments as experiments
#from influence.nlprocessor import NLProcessor
from influence.binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS
from load_heart import load_heart

import tensorflow as tf

np.random.seed(12)



In [6]:
os.chdir('/Users/mr54725/Documents/repos/INF')


In [7]:
data_sets = load_heart()

num_classes = 2

input_dim = data_sets.train.x.shape[1]
weight_decay = 0.0001
# weight_decay = 1000 / len(lr_data_sets.train.labels)
batch_size = 100
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]
max_lbfgs_iter = 10000

tf.reset_default_graph()

tf_model = BinaryLogisticRegressionWithLBFGS(
    input_dim=input_dim,
    weight_decay=weight_decay,
    max_lbfgs_iter=max_lbfgs_iter,
    num_classes=num_classes, 
    batch_size=batch_size,
    data_sets=data_sets,
    initial_learning_rate=initial_learning_rate,
    keep_probs=keep_probs,
    decay_epochs=decay_epochs,
    mini_batch=False,
    train_dir='output',
    log_dir='log',
    model_name='heart3_logreg')



Total number of parameters: 13


/Users/mr54725/anaconda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [4]:
tf_model.train()

X_train = np.copy(tf_model.data_sets.train.x)
Y_train = np.copy(tf_model.data_sets.train.labels)
X_test = np.copy(tf_model.data_sets.test.x)
Y_test = np.copy(tf_model.data_sets.test.labels) 


num_train_examples = Y_train.shape[0] 
num_flip_vals = 6
num_check_vals = 6
num_random_seeds = 40

dims = (num_flip_vals, num_check_vals, num_random_seeds, 3)
fixed_influence_loo_results = np.zeros(dims)
fixed_loss_results = np.zeros(dims)
fixed_random_results = np.zeros(dims)

flipped_results = np.zeros((num_flip_vals, num_random_seeds, 3))

orig_results = tf_model.sess.run(
    [tf_model.loss_no_reg, tf_model.accuracy_op], 
    feed_dict=tf_model.all_test_feed_dict)
            
print('Orig loss: %.5f. Accuracy: %.3f' % (orig_results[0], orig_results[1]))

for flips_idx in range(num_flip_vals):
    for random_seed_idx in range(num_random_seeds):
        
        random_seed = flips_idx * (num_random_seeds * 3) + (random_seed_idx * 2)        
        np.random.seed(random_seed)
    
        num_flips = int(num_train_examples / 20) * (flips_idx + 1)    
        idx_to_flip = np.random.choice(num_train_examples, size=num_flips, replace=False)
        Y_train_flipped = np.copy(Y_train)
        Y_train_flipped[idx_to_flip] = 1 - Y_train[idx_to_flip] 
        
        tf_model.update_train_x_y(X_train, Y_train_flipped)
        tf_model.train()        
        flipped_results[flips_idx, random_seed_idx, 1:] = tf_model.sess.run(
            [tf_model.loss_no_reg, tf_model.accuracy_op], 
            feed_dict=tf_model.all_test_feed_dict)
        print('Flipped loss: %.5f. Accuracy: %.3f' % (
                flipped_results[flips_idx, random_seed_idx, 1], flipped_results[flips_idx, random_seed_idx, 2]))
        
        train_losses = tf_model.sess.run(tf_model.indiv_loss_no_reg, feed_dict=tf_model.all_train_feed_dict)
        train_loo_influences = tf_model.get_loo_influences()

        for checks_idx in range(num_check_vals):
            np.random.seed(random_seed + 1)
            num_checks = int(num_train_examples / 20) * (checks_idx + 1)

            print('### Flips: %s, rs: %s, checks: %s' % (num_flips, random_seed_idx, num_checks))

            fixed_influence_loo_results[flips_idx, checks_idx, random_seed_idx, :], \
              fixed_loss_results[flips_idx, checks_idx, random_seed_idx, :], \
              fixed_random_results[flips_idx, checks_idx, random_seed_idx, :] \
              = experiments.test_mislabeled_detection_batch(
                tf_model, 
                X_train, Y_train,
                Y_train_flipped,
                X_test, Y_test, 
                train_losses, train_loo_influences,
                num_flips, num_checks)


np.savez(
    'output/heart_results2', 
    orig_results=orig_results,
    flipped_results=flipped_results,
    fixed_influence_loo_results=fixed_influence_loo_results,
    fixed_loss_results=fixed_loss_results,
    fixed_random_results=fixed_random_results
)


Using normal model
LBFGS training took [416] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.3592314
Train loss (w/o reg) on all data: 0.3589113
Test loss (w/o reg) on all data: 0.40642902
Train acc on all data:  0.8301886792452831
Test acc on all data:   0.8
Norm of the mean of gradients: 0.00076461677
Norm of the params: 2.5301914
Orig loss: 0.40643. Accuracy: 0.800
Using normal model
LBFGS training took [243] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.4286149
Train loss (w/o reg) on all data: 0.42844188
Test loss (w/o reg) on all data: 0.4055303
Train acc on all data:  0.8018867924528302
Test acc on all data:   0.8
Norm of the mean of gradients: 0.0010028951
Norm of the params: 1.8602495
Flipped loss: 0.40553. Accuracy: 0.800
### Flips: 10, rs: 0, checks: 10
Using normal model
LBFGS training took [252] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.39530292
Train loss (w/o reg) on all data: 0.39509252
Test loss (w/o r

LBFGS training took [245] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.41572013
Train loss (w/o reg) on all data: 0.41550335
Test loss (w/o reg) on all data: 0.37301695
Train acc on all data:  0.8160377358490566
Test acc on all data:   0.8444444444444444
Norm of the mean of gradients: 0.0030046795
Norm of the params: 2.0822592
Flipped loss: 0.37302. Accuracy: 0.844
### Flips: 10, rs: 1, checks: 10
Using normal model
LBFGS training took [235] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.38820213
Train loss (w/o reg) on all data: 0.3879434
Test loss (w/o reg) on all data: 0.39713487
Train acc on all data:  0.8349056603773585
Test acc on all data:   0.8
Norm of the mean of gradients: 0.0008716836
Norm of the params: 2.2748306
     Influence (LOO): fixed   2 labels. Loss 0.39713. Accuracy 0.800.
Using normal model
LBFGS training took [177] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.36588275
Train loss (w/o reg) on all d

### Flips: 10, rs: 2, checks: 10
Using normal model
LBFGS training took [112] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.39500993
Train loss (w/o reg) on all data: 0.39480647
Test loss (w/o reg) on all data: 0.42425647
Train acc on all data:  0.8349056603773585
Test acc on all data:   0.8
Norm of the mean of gradients: 0.0009373978
Norm of the params: 2.017247
     Influence (LOO): fixed   2 labels. Loss 0.42426. Accuracy 0.800.
Using normal model
LBFGS training took [164] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.37371653
Train loss (w/o reg) on all data: 0.37345892
Test loss (w/o reg) on all data: 0.41516155
Train acc on all data:  0.839622641509434
Test acc on all data:   0.8222222222222222
Norm of the mean of gradients: 0.003668882
Norm of the params: 2.2698874
                Loss: fixed   3 labels. Loss 0.41516. Accuracy 0.822.
Using normal model
LBFGS training took [159] iter.
After training with LBFGS: 
Train loss (w reg) on al

LBFGS training took [214] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.38210738
Train loss (w/o reg) on all data: 0.3817993
Test loss (w/o reg) on all data: 0.38626808
Train acc on all data:  0.8301886792452831
Test acc on all data:   0.8
Norm of the mean of gradients: 0.0013687053
Norm of the params: 2.4822307
     Influence (LOO): fixed   2 labels. Loss 0.38627. Accuracy 0.800.
Using normal model
LBFGS training took [197] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.38565812
Train loss (w/o reg) on all data: 0.3853438
Test loss (w/o reg) on all data: 0.4166671
Train acc on all data:  0.8254716981132075
Test acc on all data:   0.8222222222222222
Norm of the mean of gradients: 0.001692349
Norm of the params: 2.507343
                Loss: fixed   1 labels. Loss 0.41667. Accuracy 0.822.
Using normal model
LBFGS training took [196] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.3894392
Train loss (w/o reg) on all data: 0.

LBFGS training took [174] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.4136907
Train loss (w/o reg) on all data: 0.41347903
Test loss (w/o reg) on all data: 0.465453
Train acc on all data:  0.8254716981132075
Test acc on all data:   0.7777777777777778
Norm of the mean of gradients: 0.0011048338
Norm of the params: 2.0573833
              Random: fixed   0 labels. Loss 0.46545. Accuracy 0.778.
### Flips: 10, rs: 4, checks: 20
Using normal model
LBFGS training took [217] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.33503613
Train loss (w/o reg) on all data: 0.3346582
Test loss (w/o reg) on all data: 0.46266523
Train acc on all data:  0.8537735849056604
Test acc on all data:   0.7777777777777778
Norm of the mean of gradients: 0.00072214054
Norm of the params: 2.7492912
     Influence (LOO): fixed   5 labels. Loss 0.46267. Accuracy 0.778.
Using normal model
LBFGS training took [0] iter.
After training with LBFGS: 
Train loss (w reg) on all data

...

LBFGS training took [373] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.3388663
Train loss (w/o reg) on all data: 0.33851323
Test loss (w/o reg) on all data: 0.5430488
Train acc on all data:  0.8349056603773585
Test acc on all data:   0.8222222222222222
Norm of the mean of gradients: 0.00028871055
Norm of the params: 2.6572688
                Loss: fixed  32 labels. Loss 0.54305. Accuracy 0.822.
Using normal model
LBFGS training took [343] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.5583374
Train loss (w/o reg) on all data: 0.55827093
Test loss (w/o reg) on all data: 0.4955609
Train acc on all data:  0.7169811320754716
Test acc on all data:   0.7777777777777778
Norm of the mean of gradients: 0.004579562
Norm of the params: 1.1530951
              Random: fixed  17 labels. Loss 0.49556. Accuracy 0.778.
### Flips: 60, rs: 34, checks: 60
Using normal model
LBFGS training took [219] iter.
After training with LBFGS: 
Train loss (w reg) on all da

LBFGS training took [120] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.49424303
Train loss (w/o reg) on all data: 0.49416918
Test loss (w/o reg) on all data: 0.36896637
Train acc on all data:  0.7594339622641509
Test acc on all data:   0.8222222222222222
Norm of the mean of gradients: 0.006367346
Norm of the params: 1.2152979
     Influence (LOO): fixed  27 labels. Loss 0.36897. Accuracy 0.822.
Using normal model
LBFGS training took [245] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.36234972
Train loss (w/o reg) on all data: 0.3622013
Test loss (w/o reg) on all data: 0.39500153
Train acc on all data:  0.8254716981132075
Test acc on all data:   0.8
Norm of the mean of gradients: 0.0010596076
Norm of the params: 1.722847
                Loss: fixed  36 labels. Loss 0.39500. Accuracy 0.800.
Using normal model
LBFGS training took [191] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.59633034
Train loss (w/o reg) on all data:

Train loss (w reg) on all data: 0.46089283
Train loss (w/o reg) on all data: 0.46078584
Test loss (w/o reg) on all data: 0.34269813
Train acc on all data:  0.7783018867924528
Test acc on all data:   0.8444444444444444
Norm of the mean of gradients: 0.003137619
Norm of the params: 1.4627448
     Influence (LOO): fixed  27 labels. Loss 0.34270. Accuracy 0.844.
Using normal model
LBFGS training took [388] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.34031975
Train loss (w/o reg) on all data: 0.33992314
Test loss (w/o reg) on all data: 0.38809314
Train acc on all data:  0.839622641509434
Test acc on all data:   0.8
Norm of the mean of gradients: 0.0012035549
Norm of the params: 2.8164215
                Loss: fixed  38 labels. Loss 0.38809. Accuracy 0.800.
Using normal model
LBFGS training took [342] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.585047
Train loss (w/o reg) on all data: 0.58501565
Test loss (w/o reg) on all data: 0.42562184
Train

LBFGS training took [356] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.5626961
Train loss (w/o reg) on all data: 0.56264734
Test loss (w/o reg) on all data: 0.41251713
Train acc on all data:  0.7452830188679245
Test acc on all data:   0.8222222222222222
Norm of the mean of gradients: 0.00163732
Norm of the params: 0.9875246
              Random: fixed  23 labels. Loss 0.41252. Accuracy 0.822.
Using normal model
LBFGS training took [320] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.61156756
Train loss (w/o reg) on all data: 0.61153007
Test loss (w/o reg) on all data: 0.4996124
Train acc on all data:  0.6839622641509434
Test acc on all data:   0.8222222222222222
Norm of the mean of gradients: 0.004016077
Norm of the params: 0.86587846
Flipped loss: 0.49961. Accuracy: 0.822
### Flips: 60, rs: 38, checks: 10
Using normal model
LBFGS training took [97] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.5891552
Train loss (w/o re

LBFGS training took [261] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.6357958
Train loss (w/o reg) on all data: 0.63575006
Test loss (w/o reg) on all data: 0.5060192
Train acc on all data:  0.6132075471698113
Test acc on all data:   0.8
Norm of the mean of gradients: 0.0014364348
Norm of the params: 0.95633566
Flipped loss: 0.50602. Accuracy: 0.800
### Flips: 60, rs: 39, checks: 10
Using normal model
LBFGS training took [212] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.58562905
Train loss (w/o reg) on all data: 0.5855484
Test loss (w/o reg) on all data: 0.44418457
Train acc on all data:  0.6650943396226415
Test acc on all data:   0.8222222222222222
Norm of the mean of gradients: 0.0040562344
Norm of the params: 1.2701288
     Influence (LOO): fixed   6 labels. Loss 0.44418. Accuracy 0.822.
Using normal model
LBFGS training took [109] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.56933594
Train loss (w/o reg) on all d